In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

import collections
import re, string
import sys
import time
from nltk.corpus import stopwords
from wordcloud import WordCloud
from mpl_toolkits.basemap import Basemap

from subprocess import check_output
%matplotlib inline
    

**Reading JSON Function**

In [2]:
import json

def init_ds(json):
    ds= {}
    keys = json.keys()
    for k in keys:
        ds[k]= []
    return ds, keys

def read_json(file):
    dataset = {}
    keys = []
    with open(file) as file_lines:
        for count, line in enumerate(file_lines):
            data = json.loads(line.strip())
            if count ==0:
                dataset, keys = init_ds(data)
            for k in keys:
                dataset[k].append(data[k])
                
        return pd.DataFrame(dataset)

In [3]:
%%time

yelp_review= read_json('../input/yelp-dataset/yelp_academic_dataset_review.json')

CPU times: user 1min 34s, sys: 18.5 s, total: 1min 52s
Wall time: 2min 24s


**Adding Year parameter in reviews file**

In [4]:
yelp_review["date"]=  pd.to_datetime(yelp_review["date"], format='%Y-%m-%d')
yelp_review['year'] = yelp_review.date.dt.year
yelp_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,year
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4.0,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02,2014
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4.0,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25,2015
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5.0,0,0,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06,2013
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2.0,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15,2010
4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4.0,0,0,0,The food is always great here. The service fro...,2011-07-28 18:05:01,2011


**Filtering reviews only containing year 2016**

In [5]:
filters = yelp_review.year == 2016
review_2016 = yelp_review[filters]
review_2016=review_2016[['business_id']]
review_2016.head(5)




,business_id
14,mw_qxZJraNu7Q6u0GkcMew
21,MJYb0fioM-X7jc_0RIYyqQ
22,jMz_y_-cWMfiZF7Q5snE6Q
27,AW_dMex_BXFzgBJFxAjDuQ
45,H_RM2u1WWGU1HkKZrYq2Ow


In [6]:
%%time

yelp_business= read_json('../input/yelp-dataset/yelp_academic_dataset_business.json')
yelp_business.head()


CPU times: user 3.69 s, sys: 337 ms, total: 4.03 s
Wall time: 4.17 s


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,921 Pearl St,Boulder,CO,80302,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u...","Gastropubs, Food, Beer Gardens, Restaurants, B...","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'..."
1,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAtt...","Salad, Soup, Sandwiches, Delis, Restaurants, C...","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', ..."
2,bvN78flM8NLprQ1a1y5dRg,The Reclaimory,4720 Hawthorne Ave,Portland,OR,97214,45.511907,-122.613693,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Antiques, Fashion, Used, Vintage & Consignment...","{'Thursday': '11:0-18:0', 'Friday': '11:0-18:0..."
3,oaepsyvc0J17qwi8cfrOWg,Great Clips,2566 Enterprise Rd,Orange City,FL,32763,28.914482,-81.295979,3.0,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Beauty & Spas, Hair Salons",None
4,PE9uqAjdw0E4-8mjGl3wVA,Crossfit Terminus,1046 Memorial Dr SE,Atlanta,GA,30316,33.747027,-84.353424,4.0,14,1,"{'GoodForKids': 'False', 'BusinessParking': '{...","Gyms, Active Life, Interval Training Gyms, Fit...","{'Monday': '16:0-19:0', 'Tuesday': '16:0-19:0'..."


**Taking business only for state Texas and category restaurants**

In [7]:

yelp_business_1= yelp_business.dropna(subset=['categories'])
y_business_tex= yelp_business_1[yelp_business_1['state'] == 'TX']
y_business_tex= y_business_tex[y_business_tex['categories'].str.contains('Restaurants')]
filters_2016= y_business_tex.reset_index()
filters_2016.head(5)


,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,38,Xw8tuI30T-xihpzwBV-zJg,El Pollo Rey,1725 E Riverside Dr,Austin,TX,78741,30.243493,-97.730141,5.0,11,0,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Food Trucks, Restaurants, Specialty Food, Food...",None
1,41,NRPemqVb4qpWFF0Avq_6OQ,Eurasia Sushi Bar & Seafood,"7101 W Hwy 71, Ste C-13",Austin,TX,78735,30.234533,-97.877262,4.5,395,1,"{'Ambience': '{'touristy': False, 'hipster': F...","Bars, Nightlife, Cocktail Bars, Seafood, Resta...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-22:0', ..."
2,61,bRsDZ44CD3uhGnRY3NeQhQ,Wendy's,6247 Mcneil Drive,Austin,TX,78729,30.441875,-97.746581,2.0,46,1,"{'RestaurantsPriceRange2': '1', 'OutdoorSeatin...","Fast Food, Restaurants, Burgers","{'Monday': '6:30-1:0', 'Tuesday': '6:30-1:0', ..."
3,65,z-0oY7VxQMQw3JHvdPejrA,Olmecas Mexican Restaurant,2121 E Oltorf St,Austin,TX,78741,30.230016,-97.732639,3.0,21,0,"{'Ambience': '{'romantic': False, 'intimate': ...","Coffee & Tea, Tex-Mex, Restaurants, Mexican, Food","{'Monday': '7:0-23:30', 'Tuesday': '7:0-23:30'..."
4,97,gmLYiyg_hW_DVQWEPgfRQA,Viuda Bistro,108 N Main St,Buda,TX,78610,30.081542,-97.844353,4.5,21,0,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","Restaurants, American (New), Bistros","{'Thursday': '17:0-21:0', 'Friday': '17:0-21:0..."


**Take intersection of reviews_2016 and business with texas state, category restaurant.**

In [ ]:
data_2016 = pd.merge(review_2016, filters_2016, how='inner', on=['business_id'])
data_2016=data_2016[['business_id']].drop_duplicates()
opened_2016_count = data_2016.size
data_2016.size

**Filtering reviews data of 2017 from reviews file**

In [ ]:
filters1 = yelp_review.year == 2017
review_2017 = yelp_review[filters1]
review_2017=review_2017[['business_id']]
review_2017.head(10)

**Intersected data of reviews_2016 and reviews_2017**

In [ ]:
final_2017 = pd.merge(review_2017, data_2016, how='inner', on=['business_id'])
final_2017.head()

**Move business ids which are not in 2017 list to closing sheet.**

In [ ]:
closing_2016=data_2016[data_2016.business_id.isin(review_2017.business_id)==False]
closing_2016.reset_index()
closed_2016_count = closing_2016.size
closing_2016.head()





**Count Reviews of 2017 and 2016 intersected data**

In [ ]:

r_count=final_2017.assign(review_count = final_2017.groupby('business_id',sort = False).cumcount())
final_2017_count=r_count.groupby('business_id', sort = False, as_index = False).max()
final_2017_count.head(10)








**Reviews which are below 10 are moved to closed sheet, rest are moved to opened sheet.**

In [ ]:
closing_soon ={}
opened_sheet ={}

for business_id,review_count in final_2017_count.iterrows():
    if review_count['review_count']<10:
        closing_soon[review_count['business_id']] = review_count['review_count']
    else:
        opened_sheet[review_count['business_id']] = review_count['review_count']
        
closing_soon_sheet = pd.DataFrame.from_dict(data= closing_soon,orient="index")

closing_soon_sheet.reset_index(inplace=True)
closing_soon_sheet.columns = ['business_id', 'review_count']



closing_soon_sheet.head()


In [ ]:
opened_sheet_2017 = pd.DataFrame.from_dict(data= opened_sheet,orient="index")

opened_sheet_2017.reset_index(inplace=True)
opened_sheet_2017.columns = ['business_id', 'review_count']
opened_sheet_2017=opened_sheet_2017.drop_duplicates().reset_index(drop=True)[['business_id']]
opened_2017_count = opened_sheet_2017.size
opened_sheet_2017.count()

**Concat closed sheet of 2016 and closed sheet of 2017**

In [ ]:
closing_data_2016_2017=pd.concat([closing_soon_sheet, closing_2016]).drop_duplicates().reset_index(drop=True)
closing_data_2016_2017=closing_data_2016_2017[['business_id']]
closed_2017_count = closing_data_2016_2017.size
closing_data_2016_2017.head()


**Get reviews of 2018**

In [ ]:
filters2 = yelp_review.year == 2018
review_2018 = yelp_review[filters2]
review_2018=review_2018[['business_id']]
review_2018.head()

**Intersect data of reviews 2018 and opened sheet till 2017**

In [ ]:
intersect_2018 = pd.merge(review_2018, opened_sheet_2017, how='inner', on=['business_id'])
intersect_2018_1=intersect_2018[['business_id']]
intersect_2018_1.head()

**Collect business ids which were there is 2017 but not found in 2018 reviews data**

In [ ]:
difference_data_2018=opened_sheet_2017[opened_sheet_2017.business_id.isin(review_2018.business_id)==False]
difference_data_2018[['business_id']].head()

**Concat closing sheet of 2016,2017 with the business ids which were not found in 2018 but were there till 2017.**

In [ ]:

closing_sheet_2018_starting=pd.concat([closing_data_2016_2017, difference_data_2018]).drop_duplicates().reset_index(drop=True)
closing_sheet_2018_starting[['business_id']].head()

**Count reviews for only intersected data of 2018 and 2016,2017**

In [ ]:
r_count_2018=intersect_2018_1.assign(review_count = intersect_2018_1.groupby('business_id',sort = False).cumcount())
final_2018_count=r_count_2018.groupby('business_id', sort = False, as_index = False).max()
final_2018_count.head()

**Add business ids in closing sheet which have reviews less than 10.**

In [ ]:
closing_soon_2018 ={}
opened_sheet_2018 ={}

for business_id,review_count in final_2018_count.iterrows():
    if review_count['review_count']<10:
        closing_soon_2018[review_count['business_id']] = review_count['review_count']
    else:
        opened_sheet_2018[review_count['business_id']] = review_count['review_count']
        
closing_soon_sheet_2018 = pd.DataFrame.from_dict(data= closing_soon_2018,orient="index")

closing_soon_sheet_2018.reset_index(inplace=True)
closing_soon_sheet_2018.columns = ['business_id', 'review_count']



closing_soon_sheet_2018.head()

**Add business ids in open sheet which have reviews more than 10**

In [ ]:
opened_sheet_2018_1 = pd.DataFrame.from_dict(data= opened_sheet_2018,orient="index")

opened_sheet_2018_1.reset_index(inplace=True)
opened_sheet_2018_1.columns = ['business_id', 'review_count']
opened_sheet_2018_1=opened_sheet_2018_1.drop_duplicates().reset_index(drop=True)[['business_id']]
opened_2018_count = opened_sheet_2018_1.size
opened_sheet_2018_1.count()

**Merge closing sheets of starting 2018 which included all closed data till 2018 and closing sheet which have lesser reviews than 10.**

In [ ]:
closing_data_end_2018=pd.concat([closing_sheet_2018_starting, closing_soon_sheet_2018]).drop_duplicates().reset_index(drop=True)
closed_2018_count = closing_data_end_2018.size
closing_data_end_2018[['business_id']].head()

**Finding 2019 reviews data**

In [ ]:
filters3 = yelp_review.year == 2019
review_2019 = yelp_review[filters3]
review_2019=review_2019[['business_id']]
review_2019.head()

**Intersect reviews_2019 and opened sheet till 2018.**

In [ ]:
intersect_2019 = pd.merge(review_2019, opened_sheet_2018_1, how='inner', on=['business_id'])
intersect_2019_1=intersect_2019[['business_id']]
intersect_2019_1.head()

**Pick those business ids which are not there in 2019 reviews data but were there in opened sheet till 2018.**

In [ ]:
difference_data_2019=opened_sheet_2018_1[opened_sheet_2018_1.business_id.isin(review_2019.business_id)==False]
difference_data_2019[['business_id']].head()


**Merge closing sheet made till 2018 and the business ids which are not found in 2019 reviews data.**

In [ ]:
closing_sheet_2019_starting=pd.concat([closing_data_end_2018, difference_data_2019]).drop_duplicates().reset_index(drop=True)
closing_sheet_2019_starting[['business_id']].head()
     
            
    

**Count reviews of 2019 instersected data.**

In [ ]:
r_count_2019=intersect_2019_1.assign(review_count = intersect_2019_1.groupby('business_id',sort = False).cumcount())
final_2019_count=r_count_2019.groupby('business_id', sort = False, as_index = False).max()
final_2019_count.head()

**Make closing and opening sheet based on reviews threshold**

In [ ]:
closing_soon_2019 ={}
opened_sheet_2019 ={}

for business_id,review_count in final_2019_count.iterrows():
    if review_count['review_count']<10:
        closing_soon_2019[review_count['business_id']] = review_count['review_count']
    else:
        opened_sheet_2019[review_count['business_id']] = review_count['review_count']
        
closing_soon_sheet_2019 = pd.DataFrame.from_dict(data= closing_soon_2019,orient="index")

closing_soon_sheet_2019.reset_index(inplace=True)
closing_soon_sheet_2019.columns = ['business_id', 'review_count']



closing_soon_sheet_2019.head()

In [ ]:
opened_sheet_2019_1 = pd.DataFrame.from_dict(data= opened_sheet_2019,orient="index")

opened_sheet_2019_1.reset_index(inplace=True)
opened_sheet_2019_1.columns = ['business_id', 'review_count']
opened_sheet_final = opened_sheet_2019_1[['business_id']]
opened_sheet_final=opened_sheet_final.drop_duplicates().reset_index(drop=True)[['business_id']]
opened_2019_count = opened_sheet_final.size
opened_sheet_final.count()

**Merge closing sheet made from less reviews in 2019 and previous closing sheet till 2019 start.**

In [ ]:
closing_data_end_2019=pd.concat([closing_sheet_2019_starting, closing_soon_sheet_2019]).drop_duplicates().reset_index(drop=True)
closing_data_final = closing_data_end_2019[['business_id']]
closed_2019_count = closing_data_final.size
closing_data_final.count()

**Output both opened and closed final files.**

In [ ]:
closing_data_final.to_csv('/kaggle/working/closing_data_final.csv')
opened_sheet_final.to_csv('/kaggle/working/opened_sheet_final.csv')

**Plotting graph for all years 2016-2019 with their open restraunt counts.**

In [ ]:
plotdata = pd.DataFrame({
    "Opened":[opened_2016_count, opened_2017_count, opened_2018_count, opened_2019_count],
    "Closed":[closed_2016_count, closed_2017_count, closed_2018_count, closed_2019_count]
    }, 
    index=["2016", "2017", "2018", "2019"]
)
plotdata.plot(kind="bar")
plt.title("Opened and Closed data of Restaurants from 2016 to 2019")
plt.xlabel("Years")
plt.ylabel("Counts")

**Reading sheet to remove ids from 2016 closed sheet for business which closed in 2016 itself**

In [ ]:

closed_in_2016=pd.read_csv('../input/deleteclosedids/delete-closed2016.csv')
print(closed_in_2016)

**Reading already output closed sheet.**

In [ ]:
previous_closed_output=pd.read_csv('./closing_data_final.csv')
print(previous_closed_output)

**Removing ids of business which were closed in 2016 itself**

In [ ]:
removed_2016_closed_data=previous_closed_output[previous_closed_output.business_id.isin(closed_in_2016.business_id)==False]
print(removed_2016_closed_data)

In [ ]:

move_openedids_from_closedto_opened_sheet=pd.read_csv('../input/opened2019/opened-till-2019.csv')
print(move_openedids_from_closedto_opened_sheet)

**Remove opened ids from closed sheet and add in opened sheet**

In [ ]:
removed_2019_opened_data=removed_2016_closed_data[removed_2016_closed_data.business_id.isin(move_openedids_from_closedto_opened_sheet.business_id)==False]
print(removed_2019_opened_data)

In [ ]:
previous_opened_output=pd.read_csv('./opened_sheet_final.csv')
print(previous_opened_output)

**Union previous opened data with the ids coming from closed sheet which were marked as open**

In [ ]:
final_opened_data_till_2019= pd.concat([previous_opened_output, move_openedids_from_closedto_opened_sheet])
final_opened_data_till_2019.reset_index(inplace=True)
opened_total = final_opened_data_till_2019.shape[0]
final_opened_data_till_2019.count




**Adding our_open parameter as 1 in opened sheet.**

In [ ]:

final_opened_data_till_2019['our_open'] = 1
print(final_opened_data_till_2019)

In [ ]:
left_out_closed=pd.read_csv('../input/leftoutclosed/left-out-closed.csv')
left_out_closed.count

In [ ]:
removed_2019_opened_data.count

**Diffrence out closed sheet and remaining closing sheet(removed_2019_opened_data) to find the difference of one row**

In [ ]:
comparison_data =removed_2019_opened_data[removed_2019_opened_data.business_id.isin(left_out_closed.business_id)==False]
print(comparison_data)

**Adding age parameter in closing file**

In [ ]:
closing_sheet_with_age ={}


for business_id,closed_year in left_out_closed.iterrows():
    closing_sheet_with_age[closed_year['business_id']] = closed_year['closed_year'] - 2016

    
closing_sheet_with_age = pd.DataFrame.from_dict(data= closing_sheet_with_age,orient="index")

closing_sheet_with_age.reset_index(inplace=True)
closing_sheet_with_age.columns = ['business_id', 'age']
closing_total = closing_sheet_with_age.shape[0]
print(closing_total)
# closing_sheet_with_age.head()

**Adding our_open=0 in closed sheet**

In [ ]:

closing_sheet_with_age['our_open'] = 0
print(closing_sheet_with_age)

**Merging closing and open_file**

In [ ]:
final_opened_closed_data= pd.concat([closing_sheet_with_age, final_opened_data_till_2019],ignore_index=True)
final_opened_closed_data.reset_index(inplace=True)
final_opened_closed_data['age'].fillna(0, inplace=True)
final_opened_closed_data['age'] = final_opened_closed_data['age'].astype(int)
print(final_opened_closed_data)
# final_opened_closed_data[['business_id','age','our_open']].head()


**Plotting graph for opened and closed data**

In [ ]:

plotdata = pd.DataFrame(
    {"Data": [opened_total, closing_total]}, 
    index=["Total_Opened", "Total_Closed"])
# Plot a bar chart
plotdata.plot(kind="bar")
plt.xticks(rotation=30, horizontalalignment="center")
plt.title("Total count of restaurants opened and closed after preprocessing")

**Splitting categories based on Restaurant cuisine**

In [ ]:
filters_2016.is_copy=False
filters_2016.loc[filters_2016.categories.str.contains('American'),'American'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('American'),'American'] = 0
filters_2016['American'] = filters_2016['American'].astype(int)

filters_2016.loc[filters_2016.categories.str.contains('Mexican'), 'Mexican'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Mexican'),'Mexican'] = 0
filters_2016['Mexican'] = filters_2016['Mexican'].astype(int)

filters_2016.loc[filters_2016.categories.str.contains('Italian'), 'Italian'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Italian'),'Italian'] = 0
filters_2016['Italian'] = filters_2016['Italian'].astype(int)

filters_2016.loc[filters_2016.categories.str.contains('Japanese'), 'Japanese'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Japanese'),'Japanese'] = 0
filters_2016['Japanese'] = filters_2016['Japanese'].astype(int)

filters_2016.loc[filters_2016.categories.str.contains('Chinese'), 'Chinese'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Chinese'),'Chinese'] = 0
filters_2016['Chinese'] = filters_2016['Chinese'].astype(int)

filters_2016.loc[filters_2016.categories.str.contains('Thai'), 'Thai'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Thai'),'Thai'] = 0
filters_2016['Thai'] = filters_2016['Thai'].astype(int)


filters_2016.loc[filters_2016.categories.str.contains('Mediterranean'), 'Mediterranean'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Mediterranean'),'Mediterranean'] = 0
filters_2016['Mediterranean'] = filters_2016['Mediterranean'].astype(int)


filters_2016.loc[filters_2016.categories.str.contains('French'), 'French'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('French'),'French'] = 0
filters_2016['French'] = filters_2016['French'].astype(int)


filters_2016.loc[filters_2016.categories.str.contains('Vietnamese'), 'Vietnamese'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Vietnamese'),'Vietnamese'] = 0
filters_2016['Vietnamese'] = filters_2016['Vietnamese'].astype(int)


filters_2016.loc[filters_2016.categories.str.contains('Greek'),'Greek'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Greek'),'Greek'] = 0
filters_2016['Greek'] = filters_2016['Greek'].astype(int)


filters_2016.loc[filters_2016.categories.str.contains('Indian'),'Indian'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Indian'),'Indian'] = 0
filters_2016['Indian'] = filters_2016['Indian'].astype(int)


filters_2016.loc[filters_2016.categories.str.contains('Korean'),'Korean'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Korean'),'Korean'] = 0
filters_2016['Korean'] = filters_2016['Korean'].astype(int)


filters_2016.loc[filters_2016.categories.str.contains('Hawaiian'),'Hawaiian'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Hawaiian'),'Hawaiian'] = 0
filters_2016['Hawaiian'] = filters_2016['Hawaiian'].astype(int)


filters_2016.loc[filters_2016.categories.str.contains('African'),'African'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('African'),'African'] = 0
filters_2016['African'] = filters_2016['African'].astype(int)


filters_2016.loc[filters_2016.categories.str.contains('Spanish'),'Spanish'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Spanish'),'Spanish'] = 0
filters_2016['Spanish'] = filters_2016['Spanish'].astype(int)


filters_2016.loc[filters_2016.categories.str.contains('Middle Eastern'),'Middle Eastern'] = 1
filters_2016.loc[~filters_2016.categories.str.contains('Middle Eastern'),'Middle Eastern'] = 0
filters_2016['Middle Eastern'] = filters_2016['Middle Eastern'].astype(int)

filters_2016['Other'] = 0
searchfor = ['American', 'Mexican','Italian','Japanese','Chinese','Thai','Mediterranean','French','Vietnamese','Greek','Indian','Korean','Hawaiian','African','Spanish','Middle Eastern']
filters_2016.loc[~filters_2016.categories.str.contains('|'.join(searchfor)),'Other'] = 1

filters_2016.reset_index(inplace=True)
filters_2016.head()


In [ ]:
our_data=pd.merge(filters_2016, final_opened_closed_data, how='inner', on=['business_id'])
our_data.reset_index(drop=True)
our_data=our_data[['business_id','age','our_open','categories','American','Mexican','Italian','Japanese','Chinese','Thai','Mediterranean','French','Vietnamese','Greek','Indian','Korean','Hawaiian','African','Spanish','Middle Eastern','Other']]
our_data.head()

In [ ]:
our_data.to_csv('/kaggle/working/our_data.csv')